In [48]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os, os.path
import sys
sys.path.insert(0, os.getcwd()+"/../../../../libs/common")
from FileUtil import gpfs2WinPath

caldatafile = r'/gpfs/WW/BD/MXP/SHARED/SEM_IMAGE/IMEC/Case02_calaveras_v3/3Tmp/CT_KPI_test/Calaveras_v3_regular_CT_KPI_003_slope_modified_revert_all_patterns/h/cache/dummydb/result/MXP/job1/ContourSelectModelCalibration430result1/caldata.txt'
caldatafile = gpfs2WinPath(caldatafile)

df = pd.read_csv(caldatafile, sep='\s+')

tgtColName = 'UserLabel'
neighborColNames = ['NeighborOrientation', 'NeighborParalism']#, 'NeighborParalism', 'NeighborContinuity',
allColNames = ['slope', 'intensity', 'ridge_intensity', 'contrast', 'EigenRatio', 'NeighborOrientation', 'NeighborParalism']
srcColNames = ['slope', 'intensity', 'ridge_intensity', 'NeighborOrientation', 'NeighborParalism']
tgtColName = 'UserLabel'

In [49]:
df.loc[:, srcColNames].describe()

,slope,intensity,ridge_intensity,NeighborOrientation,NeighborParalism
count,3029.000000,3029.000000,3029.000000,3029.000000,3029.000000
mean,0.000437,0.493400,0.013128,0.998819,0.995971
std,0.000368,0.062304,0.005867,0.003739,0.020979
min,-0.001939,0.345614,0.000814,0.934455,0.681708
25%,0.000217,0.449102,0.009347,0.999153,0.999654
50%,0.000452,0.489985,0.013922,0.999843,0.999954
75%,0.000680,0.547742,0.017723,0.999974,0.999993
max,0.001602,0.608271,0.026195,1.000000,1.000000


In [37]:
df = df.transform(lambda x: (x - x.min()) / (x.max() - x.min()))
srcColNames = ['slope', 'intensity', 'ridge_intensity'] #, 'NeighborOrientation', 'NeighborParalism']
X = df.loc[:, srcColNames].values
y = df.loc[:, tgtColName].values

In [2]:
%matplotlib auto

# hist plot 

df2 = pd.melt(df, id_vars=tgtColName, value_vars=neighborColNames, value_name='value')
bins=np.linspace(df2.value.min(), df2.value.max(), 100)
g = sns.FacetGrid(df2, col="variable", hue=tgtColName, palette="Set1", col_wrap=3)
g.map(plt.hist, 'value', bins=bins, ec="k")
plt.yscale('log')
g.axes[-1].legend()

# scatter plot 
#sns.pointplot('NeighborOrientation', 'NeighborParalism', hue=tgtColName, data=df)

plt.show()

Using matplotlib backend: Qt5Agg


In [22]:
%matplotlib auto
def sephist(col):
    TP = df[df[tgtColName] == 0][col]
    TN = df[df[tgtColName] == 1][col]
    return TP, TN
labels = ['intensity', 'slope', 'ridge_intensity', 'contrast', 'EigenRatio']
df.loc[:, 'EigenRatio'] = df.loc[:, 'EigenRatio'].abs()
for num, alpha in enumerate(neighborColNames+labels):
    plt.subplot(2, 4, num+1)

    TP, TN = sephist(alpha)
    plt.hist((TP, TN), bins=25, alpha=0.5, label=map(''.join, zip(2*[tgtColName], 2*['=='], ['0', '1'])), color=['b', 'g'])
    #plt.hist(TP, bins=50, alpha=0.5, label=tgtColName+'==0', color='b')
    #plt.hist(TN, bins=50, alpha=0.5, label=tgtColName+'==1', color='g')
    plt.legend(loc='upper right')
    plt.title(alpha)
    plt.yscale('log')
#plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
plt.show()

Using matplotlib backend: Qt5Agg


C:\Users\peyang\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:57: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)


In [3]:
df.loc[:, tgtColName].value_counts()

1.0    2596
0.0     433
Name: UserLabel, dtype: int64

In [23]:
df.loc[df[tgtColName]==0, srcColNames].describe()

,ridge_intensity,NeighborOrientation,NeighborParalism
count,433.000000,433.000000,433.000000
mean,0.076183,0.929222,0.917139
std,0.051688,0.132003,0.156791
min,0.000000,0.000000,0.000000
25%,0.039227,0.925712,0.922903
50%,0.062372,0.977588,0.984828
75%,0.100145,0.995445,0.998136
max,0.276776,1.000000,1.000000


In [24]:
df.loc[df[tgtColName]==1, srcColNames].describe()

,ridge_intensity,NeighborOrientation,NeighborParalism
count,2596.000000,2596.000000,2596.000000
mean,0.553373,0.990775,0.999052
std,0.171287,0.018840,0.004123
min,0.100585,0.780979,0.892646
25%,0.426679,0.991966,0.999509
50%,0.581611,0.998230,0.999904
75%,0.683189,0.999676,0.999982
max,1.000000,1.000000,1.000000


In [38]:
from sklearn import svm
clf = svm.SVC(kernel='linear', class_weight='balanced') # {0: 10, 1: 1}
model = clf.fit(X, y)

modelform = pd.DataFrame(data=clf.coef_.flatten(), index=srcColNames)
modelform.loc['intercept', 0] = clf.intercept_
print(modelform)



                         0
slope             2.194072
intensity        -0.544468
ridge_intensity  11.224382
intercept        -3.765906
SVC model rms on calibration set: 0.1614967953936444
SVC model confusion matrix on calibration set:
[[ 427    6]
 [  73 2523]]
[[ 0.98614319  0.01385681]
 [ 0.02812018  0.97187982]]


In [50]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
model = clf.fit(X, y)

In [51]:
from sklearn.metrics import confusion_matrix
calcRMS = lambda y_pred, y: np.sqrt(np.mean(np.power(y_pred - y, 2)))
y_cal_pred = model.predict(X)
rms = calcRMS(y_cal_pred, y)
cm = confusion_matrix(y, y_cal_pred)
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print("DT model rms on calibration set: {}".format(rms))
print("DT model confusion matrix on calibration set:\n{}\n{}".format(cm, cm_norm))

DT model rms on calibration set: 0.0
DT model confusion matrix on calibration set:
[[ 433    0]
 [   0 2596]]
[[ 1.  0.]
 [ 0.  1.]]


In [59]:
import sys
import os.path
sys.path.insert(0, os.getcwd()+"/../../../../libs/tacx")
print(os.getcwd()+"/../../../../libs/tacx")
from SEMContour import *
sys.path.insert(0, os.getcwd()+"/../../../../libs/common")
from FileUtil import gpfs2WinPath

contourfile = r'/gpfs/WW/BD/MXP/SHARED/SEM_IMAGE/IMEC/Case02_calaveras_v3/3Tmp/CT_KPI_test/Calaveras_v3_regular_CT_KPI_003_slope_modified_revert_all_patterns/h/cache/dummydb/result/MXP/job1/ContourSelectModelCalibration430result1/461_image_contour.txt'
contourfile = gpfs2WinPath(contourfile)

class ContourAnalyzer(object):
    """docstring for ContourData"""
    def __init__(self, contourfile):
        self.__build(contourfile)

    def __build(self, contourfile):
        contour = SEMContour()
        contour.parseFile(contourfile)
        if not contour:
            sys.exit("ERROR: read in contour file %s fails\n" % contourfile)
        self.contour = contour
        self.df = contour.toDf()
# get contour data
ca = ContourAnalyzer(contourfile)
df = ca.df

C:\Localdata\D\Note\Python\apps\MXP\ContourSelect\unittest/../../../../libs/tacx


FileNotFoundError: [Errno 2] No such file or directory: '\\\\devshare-brion.asml.com\\cnfs-WW\\BD\\MXP\\SHARED\\SEM_IMAGE\\IMEC\\Case02_calaveras_v3\\3Tmp\\CT_KPI_test\\Calaveras_v3_regular_CT_KPI_003_slope_modified_revert_all_patterns\\h\\cache\\dummydb\\result\\MXP\\job1\\ContourSelectModelCalibration430result1\\461_image_contour.txt'